In [ ]:
# 건강보험심사평가원 제공 Open API 사용해서 병원 현황 데이터 조회
# - 진료과목코드 선택해서 해당 진료과목을 지원하는 병원 조회
# 테이블 만들기 - 테이블에 조회된 데이터 저장
# 자치구 또는 시도별 병원 현황 조회해서 파일로 저장

In [15]:
# 라이브러리 준비
import requests
import pandas as pd
import pymysql
import time

In [4]:
# 데이터 요청 1
api_key="R+BEwJ49XVmTutdoCAdNq4m7wjOsdFNhVtlzkxWVeZJPhwQqWbGcD7CNeQPipFzIWAanQVw/rPLsGkfcYvD4AQ=="
# url = f"http://apis.data.go.kr/B551182/hospInfoServicev2/getHospBasisList?ServiceKey={api_key}"

url = "http://apis.data.go.kr/B551182/hospInfoServicev2/getHospBasisList"
params = {
    "ServiceKey": api_key,
    "dgsbjtCd": "07" # 심장혈관흉부외과
}

resp = requests.get(url, params=params)

print(resp.content.decode('utf-8'))

<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><addr>경기도 수원시 팔달구 중부대로 93, (지동)</addr><clCd>01</clCd><clCdNm>상급종합</clCdNm><cmdcGdrCnt>0</cmdcGdrCnt><cmdcIntnCnt>0</cmdcIntnCnt><cmdcResdntCnt>0</cmdcResdntCnt><cmdcSdrCnt>0</cmdcSdrCnt><detyGdrCnt>0</detyGdrCnt><detyIntnCnt>0</detyIntnCnt><detyResdntCnt>0</detyResdntCnt><detySdrCnt>7</detySdrCnt><drTotCnt>260</drTotCnt><emdongNm>지동</emdongNm><estbDd>19670523</estbDd><hospUrl>http://www.cmcvincent.or.kr/skip.html</hospUrl><mdeptGdrCnt>1</mdeptGdrCnt><mdeptIntnCnt>1</mdeptIntnCnt><mdeptResdntCnt>9</mdeptResdntCnt><mdeptSdrCnt>242</mdeptSdrCnt><pnursCnt>0</pnursCnt><postNo>16247</postNo><sgguCd>310603</sgguCd><sgguCdNm>수원팔달구</sgguCdNm><sidoCd>310000</sidoCd><sidoCdNm>경기</sidoCdNm><telno>031-1577-8588</telno><XPos>127.0274271</XPos><YPos>37.2779855</YPos><yadmNm>가톨릭대학교 성빈센트병원</yadmNm><ykiho>JDQ4MTYyMiM1MSMkMSMkMCMkODkkMzgxMzU

In [5]:
from xml.etree import ElementTree as ET

result = ET.fromstring(resp.content.decode("utf-8")) # xml string -> object tree 변환

In [ ]:
# xml을 읽어서 만든 객체 트리 구조 확인
print(result.tag)

for c1 in result: # result 태그 하위 요소를 순회
    print(" " * 2, c1.tag)
    for c2 in c1: # c1 태그 하위 요소를 순회
        print(" " * 4, c2.tag)
        for c3 in c2: # c2 태그 하위 요소를 순회
            print(" " * 6, c3.tag)

response
   header
     resultCode
     resultMsg
   body
     items
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
     numOfRows
     pageNo
     totalCount


In [12]:
# 실제 데이터가 있는 item 태그를 읽어서 데이터 뽑기

items = result.find('body').find('items').findall('item') # find는 1개 조회, findall 여러 개 조회

columns = ['yadmNm', 'clCdNm', 'sidoCdNm', 'sgguCdNm', 'emdongNm', 'addr', 'hospUrl']

data = []
for item in items:
    row = {}
    for c in item:
        if c.tag in columns:
            row.update({c.tag:c.text})
    data.append(row)

In [ ]:
data

[{'addr': '경기도 수원시 팔달구 중부대로 93, (지동)',
  'clCdNm': '상급종합',
  'emdongNm': '지동',
  'hospUrl': 'http://www.cmcvincent.or.kr/skip.html',
  'sgguCdNm': '수원팔달구',
  'sidoCdNm': '경기',
  'yadmNm': '가톨릭대학교 성빈센트병원'},
 {'addr': '인천광역시 부평구 동수로 56, (부평동)',
  'clCdNm': '상급종합',
  'emdongNm': '부평동',
  'hospUrl': 'http://www.cmcism.or.kr/',
  'sgguCdNm': '인천부평구',
  'sidoCdNm': '인천',
  'yadmNm': '가톨릭대학교인천성모병원'},
 {'addr': '강원특별자치도 강릉시 사천면 방동길 38, (사천면)',
  'clCdNm': '상급종합',
  'emdongNm': '사천면',
  'hospUrl': 'http://www.gnah.co.kr',
  'sgguCdNm': '강릉시',
  'sidoCdNm': '강원',
  'yadmNm': '강릉아산병원'},
 {'addr': '서울특별시 종로구 새문안로 29, (평동)',
  'clCdNm': '상급종합',
  'emdongNm': '평동',
  'hospUrl': 'http://www.kbsmc.co.kr',
  'sgguCdNm': '종로구',
  'sidoCdNm': '서울',
  'yadmNm': '강북삼성병원'},
 {'addr': '서울특별시 광진구 능동로 120-1, (화양동)',
  'clCdNm': '상급종합',
  'emdongNm': '화양동',
  'hospUrl': 'http://www.kuh.ac.kr',
  'sgguCdNm': '광진구',
  'sidoCdNm': '서울',
  'yadmNm': '건국대학교병원'},
 {'addr': '대구광역시 중구 동덕로 130, (삼덕동2가, 경북대학교병원)',
  'clC

In [14]:
df_hospitals = pd.DataFrame(data)
df_hospitals.head()

,addr,clCdNm,emdongNm,hospUrl,sgguCdNm,sidoCdNm,yadmNm
0,"경기도 수원시 팔달구 중부대로 93, (지동)",상급종합,지동,http://www.cmcvincent.or.kr/skip.html,수원팔달구,경기,가톨릭대학교 성빈센트병원
1,"인천광역시 부평구 동수로 56, (부평동)",상급종합,부평동,http://www.cmcism.or.kr/,인천부평구,인천,가톨릭대학교인천성모병원
2,"강원특별자치도 강릉시 사천면 방동길 38, (사천면)",상급종합,사천면,http://www.gnah.co.kr,강릉시,강원,강릉아산병원
3,"서울특별시 종로구 새문안로 29, (평동)",상급종합,평동,http://www.kbsmc.co.kr,종로구,서울,강북삼성병원
4,"서울특별시 광진구 능동로 120-1, (화양동)",상급종합,화양동,http://www.kuh.ac.kr,광진구,서울,건국대학교병원


In [19]:
# 데이터 조회 2. 전체 데이터 조회

api_key = "R+BEwJ49XVmTutdoCAdNq4m7wjOsdFNhVtlzkxWVeZJPhwQqWbGcD7CNeQPipFzIWAanQVw/rPLsGkfcYvD4AQ=="
url = "http://apis.data.go.kr/B551182/hospInfoServicev2/getHospBasisList"

columns = ['yadmNm', 'clCdNm', 'sidoCdNm', 'sgguCdNm', 'emdongNm', 'addr', 'hospUrl']
cnt_per_page = 100 # 한 번 요청할 때 받아올 데이터 갯수
page = 1
data = []
while True:    
    params = {
        "ServiceKey": api_key,
        "dgsbjtCd": "07", # 심장혈관흉부외과
        "pageNo": page, 
        "numOfRows": cnt_per_page
    }
    resp = requests.get(url, params)
    parsed_xml = ET.fromstring(resp.content.decode('utf-8'))
    items = parsed_xml.find('body').find('items').findall("item")
    for item in items:
        row = {}
        for child in item:
            if child.tag in columns:
                row.update({child.tag: child.text})
        data.append(row)

    total_cnt = int(parsed_xml.find('body').find('totalCount').text) # 전체 데이터 갯수 읽기
    if total_cnt <= len(data):
        break

    page += 1

    time.sleep(0.1) # 여기서 0.1초 실행 중지


In [20]:
len(data)

1137